# Entrenamiento Etapa 1: Predicción Binaria (¿Habrá Réplica Fuerte?)

## Objetivo
Entrenar dos modelos (Decision Tree y kNN) para predecir si habrá una réplica importante después de un terremoto principal.

**Target:** `existe_replica_fuerte` (1 = Sí habrá, 0 = No habrá)

**Algoritmos:**
- Decision Tree (Árbol de decisión)
- kNN (K-vecinos más cercanos)

**Validación:** Temporal (entrenar con eventos antiguos, testear con recientes)

## Estrategia de Desbalance de Clases

**Decisión:** NO usar SMOTE ni oversampling manual.

**Método:** Usar `class_weight='balanced'` en modelos que lo soporten.

**Razón:**
- SMOTE puede crear datos sintéticos que no representan sismos reales.
- `class_weight='balanced'` ajusta automáticamente los pesos para dar más importancia a la clase minoritaria (réplicas fuertes) sin inventar datos.
- Para kNN (que no tiene class_weight), usamos `weights='distance'` que da más peso a vecinos cercanos.

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# Utilidades propias
import sys
sys.path.append('..')  # Para importar desde carpeta superior si fuera necesario
from utils_validacion import (
    limpiar_columnas_vacias,
    split_temporal,
    preparar_train_test,
    obtener_columnas_numericas
)

# Configuración visual
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print('Librerías importadas correctamente.')

Librerías importadas correctamente.


## 1. Cargar y Limpiar Datos

In [4]:
# Cargar dataset maestro
master = pd.read_csv('../3ra_etapa_preprocesamiento/seismic_features_fusion_final.csv')

# Filtrar solo mainshocks (Etapa 1)
etapa1 = master[master['es_mainshock'] == 1].copy()

print(f'Dataset Etapa 1: {etapa1.shape[0]} terremotos principales (mainshocks)')
print(f'Columnas: {etapa1.shape[1]}')

# Limpiar columnas totalmente vacías
etapa1, cols_eliminadas = limpiar_columnas_vacias(etapa1)

# Verificar distribución del target
print('\nDistribución de réplicas fuertes:')
print(etapa1['existe_replica_fuerte'].value_counts())
print(f"Porcentaje con réplica: {etapa1['existe_replica_fuerte'].mean()*100:.1f}%")

Dataset Etapa 1: 236 terremotos principales (mainshocks)
Columnas: 29
[Limpieza] Columnas 100% NaN eliminadas: ['similitud_promedio_vecinos', 'conflicto_modelos']

Distribución de réplicas fuertes:
existe_replica_fuerte
0    211
1     25
Name: count, dtype: int64
Porcentaje con réplica: 10.6%


## 2. División Temporal (Train/Test)

**Nota:** No usamos oversampling (SMOTE) ni balanceo manual de clases. En su lugar, configuramos `class_weight='balanced'` en los modelos para que automáticamente den más peso a la clase minoritaria.

In [ ]:
# Split temporal: 70% entrenamiento (eventos antiguos), 30% prueba (eventos recientes)
train, test = split_temporal(etapa1, col_fecha='Date(UTC)', porcentaje_train=0.7)

# Verificar balance en cada conjunto
print('\nBalance en Train:')
print(train['existe_replica_fuerte'].value_counts(normalize=True))
print('\nBalance en Test:')
print(test['existe_replica_fuerte'].value_counts(normalize=True))

## 3. Preparar Features y Target

In [ ]:
# Obtener columnas numéricas (excluyendo targets y fecha)
columnas_features = obtener_columnas_numericas(etapa1)

print(f'\nFeatures seleccionadas: {len(columnas_features)}')
print(columnas_features[:10], '...')  # Mostrar primeras 10

# Preparar conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test, scaler = preparar_train_test(
    train, test,
    columnas_features=columnas_features,
    col_target='existe_replica_fuerte'
)

print(f'\nFormas finales:')
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'X_test: {X_test.shape}, y_test: {y_test.shape}')

## 4. Entrenar Decision Tree (Árbol de Decisión)

In [ ]:
# Crear y entrenar modelo Decision Tree
dt_model = DecisionTreeClassifier(
    max_depth=10,           # Profundidad máxima (evita sobreajuste)
    min_samples_split=10,   # Mínimo de muestras para dividir un nodo
    min_samples_leaf=5,     # Mínimo de muestras en una hoja
    class_weight='balanced', # Ajusta pesos para manejar desbalance (en vez de SMOTE/oversampling)
    random_state=42
)

print('Entrenando Decision Tree...')
dt_model.fit(X_train, y_train)

# Predicciones
y_pred_dt = dt_model.predict(X_test)

print('✓ Modelo Decision Tree entrenado.')

## 5. Entrenar kNN (K-Vecinos Más Cercanos)

In [ ]:
# Crear y entrenar modelo kNN
# Nota: kNN no tiene class_weight, pero usamos weights='distance' 
# que da más importancia a vecinos cercanos (ayuda con desbalance)
knn_model = KNeighborsClassifier(
    n_neighbors=5,      # Número de vecinos a consultar
    weights='distance', # Vecinos más cercanos tienen más peso (alternativa a class_weight)
    metric='euclidean'  # Métrica de distancia
)

print('Entrenando kNN...')
knn_model.fit(X_train, y_train)

# Predicciones
y_pred_knn = knn_model.predict(X_test)

print('✓ Modelo kNN entrenado.')

## 6. Evaluar Modelos: Métricas

In [ ]:
# Función para calcular y mostrar métricas
def evaluar_modelo(y_true, y_pred, nombre_modelo):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f'\n═══ Métricas: {nombre_modelo} ═══')
    print(f'Accuracy (Aciertos totales):  {acc:.3f} ({acc*100:.1f}%)')
    print(f'Precision (¿Cuántas alarmas son correctas?): {prec:.3f}')
    print(f'Recall (¿Cuántas réplicas reales detecta?):   {rec:.3f}')
    print(f'F1-Score (Balance precision/recall):         {f1:.3f}')
    
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1}

# Evaluar Decision Tree
metricas_dt = evaluar_modelo(y_test, y_pred_dt, 'Decision Tree')

# Evaluar kNN
metricas_knn = evaluar_modelo(y_test, y_pred_knn, 'kNN')

## 7. Matrices de Confusión

In [ ]:
# Función para graficar matriz de confusión
def plot_confusion_matrix(y_true, y_pred, titulo):
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Réplica', 'Sí Réplica'],
                yticklabels=['No Réplica', 'Sí Réplica'])
    plt.title(titulo)
    plt.ylabel('Real')
    plt.xlabel('Predicción')
    plt.tight_layout()
    plt.show()
    
    # Interpretar matriz
    tn, fp, fn, tp = cm.ravel()
    print(f'\nInterpretación:')
    print(f'  Verdaderos Negativos (TN): {tn} - Correctamente predijo "No habrá réplica"')
    print(f'  Falsos Positivos (FP):     {fp} - Dijo "Sí" pero no hubo réplica (falsa alarma)')
    print(f'  Falsos Negativos (FN):     {fn} - Dijo "No" pero sí hubo réplica (¡PELIGROSO!)')
    print(f'  Verdaderos Positivos (TP): {tp} - Correctamente predijo "Sí habrá réplica"')

# Graficar Decision Tree
plot_confusion_matrix(y_test, y_pred_dt, 'Matriz de Confusión - Decision Tree')

# Graficar kNN
plot_confusion_matrix(y_test, y_pred_knn, 'Matriz de Confusión - kNN')

## 8. Comparación de Modelos

In [ ]:
# Crear DataFrame comparativo
comparacion = pd.DataFrame({
    'Decision Tree': metricas_dt,
    'kNN': metricas_knn
})

print('\n═══ Comparación de Modelos ═══')
print(comparacion.T.round(3))

# Gráfico de barras
comparacion.T.plot(kind='bar', figsize=(10, 5))
plt.title('Comparación de Métricas: Decision Tree vs kNN')
plt.ylabel('Valor')
plt.xlabel('Modelo')
plt.xticks(rotation=0)
plt.legend(title='Métrica')
plt.tight_layout()
plt.show()

## 9. Conclusiones Etapa 1

### Interpretación de Resultados

**Accuracy:** Porcentaje total de aciertos (incluye predicciones correctas de "No" y "Sí").

**Precision:** De las veces que el modelo dijo "Sí habrá réplica", ¿cuántas acertó?
- Alta precision = Pocas falsas alarmas.

**Recall:** De todas las réplicas reales, ¿cuántas detectó?
- Alto recall = Detecta la mayoría de réplicas (crítico para seguridad).

**F1-Score:** Balance entre precision y recall.

### ¿Cuál Modelo Usar?

- Si **Recall** es más importante (no queremos perder réplicas reales) → Elegir modelo con mayor Recall.
- Si **Precision** es más importante (evitar falsas alarmas) → Elegir modelo con mayor Precision.
- Para balance → Elegir modelo con mayor **F1-Score**.

### Próximos Pasos

1. **Ajustar parámetros** del mejor modelo (profundidad del árbol, número de vecinos).
2. **Probar variantes de features** (todas vs PCA vs top correlación).
3. **Pasar a Etapa 2:** Entrenar modelos para predecir ventana temporal (0-24h, 24-72h, >72h).

In [ ]:
# Guardar modelos entrenados (opcional)
import joblib

joblib.dump(dt_model, 'decision_tree_etapa1.pkl')
joblib.dump(knn_model, 'knn_etapa1.pkl')
joblib.dump(scaler, 'scaler_etapa1.pkl')

print('\n✓ Modelos guardados:')
print('  - decision_tree_etapa1.pkl')
print('  - knn_etapa1.pkl')
print('  - scaler_etapa1.pkl')